# Train Using MLM

In this notebook we'll take a look at the logic behind fine-tuning a model using masked-language modelling (MLM).

First we'll import all we need.

In [1]:

from transformers import BertTokenizer, BertForMaskedLM
import torch

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
outputs.keys()

odict_keys(['logits'])

This returns just our MLM output logits.

In [4]:
outputs.logits.shape

torch.Size([1, 62, 30522])

To identify the token position where we have **\[MASK\]** tokens we can check the `inputs` tensor for tokens matching *103* (eg MASK).

In [6]:
mask_pos = torch.flatten((inputs.input_ids[0] == 103).nonzero()).tolist()
mask_pos

[9, 43]

It is for these two positions that we must calculate the loss for when training our model. How does that work? Well, we compare the `inputs` at those two positions, to the predicted `outputs` at those two positions - converted to one-hot encoding and probability distribution respectively.

To convert the `inputs` tokens to one-hot encodings we need the vocab dictionary length, and add a **1** at the position given by the token value.

In [7]:
vocab_size = len(tokenizer.get_vocab())
vocab_size

30522

Which aligns to our `outputs.logits.shape` tensor shape too:

In [8]:
outputs.logits.shape

torch.Size([1, 62, 30522])

In reality, we must mask our tokens randomly, after which we feed in the original (unmasked) *token_ids* into to model as *labels*, and keep *token_ids* as the new masked tensor.

To test this, let's unmask our text first.

In [11]:
text = ("After Abraham Lincoln won the November 1860 presidential election on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces attacked Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

And now convert our text using the `tokenizer`:

In [12]:
inputs = tokenizer(text, return_tensors='pt')

In [13]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Now we want to mask a number of tokens in the *input_ids* tensor, the original BERT model was pretrained using a masking probability of 15%, alongside a few additional rules. We will create a similiar, but simpler implementation by taking the primary 15% masking functionality.

In [14]:
# create random array of floats in equal dimension to input_ids
rand = torch.rand(inputs.input_ids.shape)
rand

tensor([[0.0737, 0.5543, 0.4369, 0.0838, 0.6557, 0.6418, 0.6419, 0.4008, 0.2343,
         0.6129, 0.2527, 0.8367, 0.1025, 0.7300, 0.7682, 0.3358, 0.6097, 0.1676,
         0.2715, 0.5524, 0.7312, 0.2514, 0.3926, 0.7550, 0.1709, 0.7833, 0.1288,
         0.3905, 0.1689, 0.0717, 0.2149, 0.2298, 0.8954, 0.9815, 0.7814, 0.4914,
         0.7294, 0.2910, 0.9615, 0.2536, 0.6677, 0.4990, 0.0447, 0.9965, 0.8923,
         0.4549, 0.7481, 0.8426, 0.1205, 0.8348, 0.5262, 0.1993, 0.8357, 0.7831,
         0.0974, 0.4800, 0.6747, 0.5037, 0.6679, 0.6960, 0.9543, 0.7409]])

In [15]:
# where the random array is less than 0.15, we set true, these will be our MASK tokens
mask_arr = rand < 0.15
mask_arr

tensor([[ True, False, False,  True, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True, False, False,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False,  True, False,
         False, False, False, False,  True, False, False, False, False, False,
         False, False]])

In [16]:
# extract the True value index positions
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

[0, 3, 12, 26, 29, 42, 48, 54]

We then select these *True* value positions in the *input_ids* tensor, and change their values to *103* - we need to copy the original tensor first though, to create a new one called *labels*:

In [17]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

And now use `mask_arr` to mask ~15% of our values:

In [18]:
# We want to mask the following input IDs
inputs.input_ids[0, selection]

tensor([ 101, 5367, 3424, 1996, 2433, 2749, 2148, 3204])

In [19]:
# Mask with the 103 mask token
inputs.input_ids[0, selection] = 103
inputs

{'input_ids': tensor([[  103,  2044,  8181,   103,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,   103,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,   103,  2406,  2000,   103,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,   103,  4457,  3481,  7680,  3334,  1999,   103,  3792,
          1010,  2074,  2058,  1037,   103,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

We should notice here that we have a slight problem, our CLS token has been replaced by a MASK token - we don't want to do this, nor do we want to replace any other special tokens. In this case, we have only two special tokens, CLS and SEP at the beginning and end of our sequence respectively. So the solution here is easy, we add another condition when creating our Boolean mask tensor `mask_arr`. This condition should check the original `inputs.input_ids`, where a value is **not** equal to *101* or *102* (*CLS* and *SEP* respectively) then we set *True*:

In [20]:
# reinitialize inputs.input_ids first (to remove mask)
inputs = tokenizer(text, return_tensors='pt')
# and copy to labels tensor
inputs['labels'] = inputs.input_ids.detach().clone()
# now test the new conditional logic
(inputs.input_ids != 101) * (inputs.input_ids != 102)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False]])

And add this logic to our `mask_arr`:

In [21]:
# where the random array is less than 0.15, we set true, these will be our MASK tokens
# BUT - we must also add our new conditions too
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False,  True, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True, False, False,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False,  True, False,
         False, False, False, False,  True, False, False, False, False, False,
         False, False]])

In [22]:
inputs.input_ids

tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]])

And run through the same steps again.

In [23]:
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()  # create selection from mask_arr
inputs.input_ids[0, selection] = 103  # apply selection index to inputs.input_ids, adding MASK tokens
inputs

{'input_ids': tensor([[  101,  2044,  8181,   103,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,   103,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,   103,  2406,  2000,   103,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,   103,  4457,  3481,  7680,  3334,  1999,   103,  3792,
          1010,  2074,  2058,  1037,   103,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

Now we're ready. We pass these inputs into our model.

In [24]:
outputs = model(**inputs)

And we'll find that we now have an additional output tensor, the *loss* tensor.

In [25]:
outputs.keys()

odict_keys(['loss', 'logits'])

In [27]:
# This is the loss we will train our MLM model on:
outputs.loss

tensor(0.6065, grad_fn=<NllLossBackward0>)

It is this loss that we will be training on.